# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### --PENDING-- US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats --PENDING--
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats --PENDING--

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'07-29-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'07-29-2020'

In [4]:
# Check out the first 5 rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,45001.0,Abbeville,South Carolina,US,2020-07-30 04:35:05,34.223334,-82.461707,279,6,0,273.0,"Abbeville, South Carolina, US",1137.521915,2.150538
1,22001.0,Acadia,Louisiana,US,2020-07-30 04:35:05,30.295065,-92.414197,2266,70,0,2196.0,"Acadia, Louisiana, US",3652.187928,3.089144
2,51001.0,Accomack,Virginia,US,2020-07-30 04:35:05,37.767072,-75.632346,1072,15,0,1057.0,"Accomack, Virginia, US",3317.242233,1.399254
3,16001.0,Ada,Idaho,US,2020-07-30 04:35:05,43.452658,-116.241552,7515,53,0,7462.0,"Ada, Idaho, US",1560.465710,0.705256
4,19001.0,Adair,Iowa,US,2020-07-30 04:35:05,41.330756,-94.471059,20,0,0,20.0,"Adair, Iowa, US",279.642058,0.000000


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/20/20,7/21/20,7/22/20,7/23/20,7/24/20,7/25/20,7/26/20,7/27/20,7/28/20,7/29/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,35526,35615,35727,35928,35981,36036,36157,36263,36368,36471
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,4171,4290,4358,4466,4570,4637,4763,4880,4997,5105
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,23691,24278,24872,25484,26159,26764,27357,27973,28615,29229


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/20/20,7/21/20,7/22/20,7/23/20,7/24/20,7/25/20,7/26/20,7/27/20,7/28/20,7/29/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,1185,1186,1190,1211,1225,1248,1259,1269,1270,1271
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,113,117,120,123,128,134,138,144,148,150
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,1087,1100,1111,1124,1136,1146,1155,1163,1174,1186


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/20/20,7/21/20,7/22/20,7/23/20,7/24/20,7/25/20,7/26/20,7/27/20,7/28/20,7/29/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,23741,23741,23924,24550,24602,24793,25180,25198,25358,25389
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,2352,2397,2463,2523,2608,2637,2682,2745,2789,2830
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,16400,16646,16983,17369,17369,18076,18088,18837,19233,19592


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,7/20/20,7/21/20,7/22/20,7/23/20,7/24/20,7/25/20,7/26/20,7/27/20,7/28/20,7/29/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,319,319,330,337,337,337,337,349,351,354
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,38,38,38,38,38,40,40,40,40,42


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,7/20/20,7/21/20,7/22/20,7/23/20,7/24/20,7/25/20,7/26/20,7/27/20,7/28/20,7/29/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,5,5,5,5,5,5,5,5,5,5
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,2,2,2,2,2,2,2,2,2,2


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
513,1023.0,Choctaw,Alabama,US,2020-07-30 04:35:05,32.022273,-88.265644,265,12,0,253.0,"Choctaw, Alabama, US",2105.012312,4.528302
355,1015.0,Calhoun,Alabama,US,2020-07-30 04:35:05,33.774837,-85.826304,1253,6,0,1247.0,"Calhoun, Alabama, US",1102.944413,0.478851
1808,1091.0,Marengo,Alabama,US,2020-07-30 04:35:05,32.247676,-87.787962,496,13,0,483.0,"Marengo, Alabama, US",2629.486296,2.620968


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,4426982,150713,1389425
Brazil,2552265,90134,1922802
India,1581963,34955,1019735


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,4426982,150713,1389425,2886844,2020-07-30 04:35:05,37.904997,-91.310033
Brazil,2552265,90134,1922802,539329,2020-07-30 04:35:05,-12.669522,-48.480493
India,1581963,34955,1019735,527273,2020-07-30 04:35:05,23.088275,81.806127
Russia,827509,13650,619204,194655,2020-07-30 04:35:05,54.546312,62.120860
South Africa,471123,7497,297967,165659,2020-07-30 04:35:05,-30.559500,22.937500


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,484913,8908,0
Florida,451423,6333,0
Texas,418995,6193,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,484913,8908,0,476005,2020-07-30 04:35:05,37.843962,-120.728594
Florida,451423,6333,0,445090,2020-07-30 04:35:05,28.940755,-82.700744
Texas,418995,6193,0,412802,2020-07-30 04:35:05,31.647790,-98.615963
New York,413593,32658,0,380935,2020-07-30 04:35:05,42.671593,-75.579694
New Jersey,180600,15798,0,164802,2020-07-30 04:35:05,40.286441,-74.622450


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
New York,New York City,224551,23512,0
California,Los Angeles,183456,4518,0
Arizona,Maricopa,113148,1915,0
Florida,Miami-Dade,113143,1455,0
Illinois,Cook,104001,4864,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
New York,New York City,224551,23512,0,201039,2020-07-30 04:35:05,40.767273,-73.971526,36061.0
California,Los Angeles,183456,4518,0,178938,2020-07-30 04:35:05,34.308284,-118.228241,6037.0
Arizona,Maricopa,113148,1915,0,111233,2020-07-30 04:35:05,33.348359,-112.491815,4013.0
Florida,Miami-Dade,113143,1455,0,111688,2020-07-30 04:35:05,25.611236,-80.551706,12086.0
Illinois,Cook,104001,4864,0,99137,2020-07-30 04:35:05,41.841448,-87.816588,17031.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-27,36263,4880,27973,907,950,86,167416,37390,15303,20558,...,301708,1202,21209,15988,431,10621,10,1691,4552,2704
2020-07-28,36368,4997,28615,907,1000,86,173355,37629,15582,20677,...,302261,1218,21699,16571,446,10938,10,1703,5002,2817
2020-07-29,36471,5105,29229,918,1078,91,178996,37937,16298,20850,...,303063,1237,22585,17158,459,11284,10,1711,5249,2879


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-27,1269,144,1163,52,41,3,3059,711,167,713,...,45844,35,121,146,0,78,1,483,140,36
2020-07-28,1270,148,1174,52,47,3,3179,719,176,713,...,45963,35,124,151,0,79,1,484,142,40
2020-07-29,1271,150,1186,52,48,3,3288,723,189,716,...,46046,35,131,156,0,80,1,485,146,41


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-27,25198,2745,18837,803,242,65,72575,26665,9311,18246,...,1437,951,11674,9959,365,3752,8,833,2815,542
2020-07-28,25358,2789,19233,803,266,65,75083,27357,9617,18379,...,1438,958,12026,10195,369,3752,8,840,3195,604
2020-07-29,25389,2830,19592,804,301,67,77855,27824,9619,18528,...,1438,967,12937,10421,369,4833,8,849,3285,887


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,7/20/20,7/21/20,7/22/20,7/23/20,7/24/20,7/25/20,7/26/20,7/27/20,7/28/20,7/29/20
130,84001097,US,USA,840,1097.0,Mobile,Alabama,US,30.784723,-88.208424,...,6150,6279,6443,6797,6901,7096,7193,7361,7495,7683
109,84001055,US,USA,840,1055.0,Etowah,Alabama,US,34.045673,-86.040519,...,1356,1422,1440,1495,1546,1615,1643,1690,1723,1743
110,84001057,US,USA,840,1057.0,Fayette,Alabama,US,33.720769,-87.738866,...,121,122,124,128,132,134,138,144,151,156
111,84001059,US,USA,840,1059.0,Franklin,Alabama,US,34.442353,-87.842895,...,1027,1038,1044,1051,1064,1075,1078,1098,1105,1117
112,84001061,US,USA,840,1061.0,Geneva,Alabama,US,31.093890,-85.835728,...,146,150,158,167,178,183,186,192,194,198


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-07-27,81115,2623,0,163827,39447,458121,44547,48983,14406,49,...,96489,400336,38409,1402,364,86072,53321,5999,49417,2520
2020-07-28,82366,2730,0,165934,40181,470762,45296,49077,14476,49,...,99044,408493,38855,1405,375,86994,54205,6120,50179,2589
2020-07-29,83782,2798,0,168273,40968,484913,45778,49540,14602,49,...,100822,418995,39194,1406,385,87993,54985,6277,51049,2628


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-07-27         932    2727     529  327    601     409    718    1157   
2020-07-28         941    2793     535  330    607     422    722    1206   
2020-07-29         949    2865     538  336    623     425    728    1253   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-07-27          779      205  ...     101      5       43       30   
2020-07-28          790      206  ...     107      5       44       31   
2020-07-29          796      211  ...     108      5       45       31   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-07-27            221   292   235          0       45      5  
2020-07-28            228   311   243          0       45      5  
2020-07-29            234   318   245          0       45      5  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-07-27,1491,21,0,3304,408,8494,1799,4418,579,0,...,978,5713,281,56,7,2082,1518,104,893,25
2020-07-28,1491,22,0,3408,428,8679,1807,4423,580,0,...,999,5913,286,56,7,2095,1548,108,906,26
2020-07-29,1538,22,0,3454,434,8908,1822,4425,581,0,...,1020,6193,292,56,8,2125,1555,111,911,26


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-07-27          20      17       4    2      1      11     35       6   
2020-07-28          20      17       4    2      1      11     35       6   
2020-07-29          20      20       4    2      3      11     35       6   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-07-27           38        7  ...       0      0        0        0   
2020-07-28           38        7  ...       0      0        0        0   
2020-07-29           38        7  ...       0      0        0        0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-07-27              0     0     0         24        0      0  
2020-07-28              0     0     0         25        0      0  
2020-07-29              0     0     0         25        0      0  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-27,0.002932,0.024564,0.022517,0.011148,0.019313,0.04878,0.030087,0.001956,0.024640,0.004201,...,0.002286,0.008389,0.033023,0.033952,0.026190,0.014519,0.0,0.005949,0.015845,0.076433
2020-07-28,0.002896,0.023975,0.022951,0.000000,0.052632,0.00000,0.035475,0.006392,0.018232,0.005789,...,0.001833,0.013311,0.023103,0.036465,0.034803,0.029847,0.0,0.007096,0.098858,0.041790
2020-07-29,0.002832,0.021613,0.021457,0.012128,0.078000,0.05814,0.032540,0.008185,0.045950,0.008367,...,0.002653,0.015599,0.040831,0.035423,0.029148,0.031633,0.0,0.004698,0.049380,0.022009


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-27,0.007943,0.043478,0.006926,0.0,0.025000,0.0,0.040830,0.008511,0.037267,0.001404,...,0.000153,0.029412,0.043103,0.028169,NaN,0.026316,0.0,0.008351,0.007194,0.058824
2020-07-28,0.000788,0.027778,0.009458,0.0,0.146341,0.0,0.039229,0.011252,0.053892,0.000000,...,0.002596,0.000000,0.024793,0.034247,NaN,0.012821,0.0,0.002070,0.014286,0.111111
2020-07-29,0.000787,0.013514,0.010221,0.0,0.021277,0.0,0.034288,0.005563,0.073864,0.004208,...,0.001806,0.000000,0.056452,0.033113,NaN,0.012658,0.0,0.002066,0.028169,0.025000


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-27,0.000715,0.023490,0.041409,0.000000,0.000000,0.083333,0.029170,0.007062,0.014934,0.002032,...,0.002092,0.003165,0.051238,0.021855,0.000000,0.000000,0.0,0.045169,0.197872,0.046332
2020-07-28,0.006350,0.016029,0.021022,0.000000,0.099174,0.000000,0.034557,0.025952,0.032864,0.007289,...,0.000696,0.007361,0.030152,0.023697,0.010959,0.000000,0.0,0.008403,0.134991,0.114391
2020-07-29,0.001222,0.014701,0.018666,0.001245,0.131579,0.030769,0.036919,0.017071,0.000208,0.008107,...,0.000000,0.009395,0.075753,0.022168,0.000000,0.288113,0.0,0.010714,0.028169,0.468543


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-07-27,0.022965,0.038812,NaN,0.011190,0.021334,0.012897,0.005167,0.004244,0.008118,0.0,...,0.027178,0.015865,0.011482,0.001429,0.034091,0.017797,0.013033,0.018679,0.012083,0.018182
2020-07-28,0.015423,0.040793,NaN,0.012861,0.018607,0.027593,0.016814,0.001919,0.004859,0.0,...,0.026480,0.020375,0.011612,0.002140,0.030220,0.010712,0.016579,0.020170,0.015420,0.027381
2020-07-29,0.017192,0.024908,NaN,0.014096,0.019586,0.030060,0.010641,0.009434,0.008704,0.0,...,0.017952,0.025709,0.008725,0.000712,0.026667,0.011484,0.014390,0.025654,0.017338,0.015064


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-07-27      0.025303  0.023265  0.011472  0.031546  0.039792  0.002451   
2020-07-28      0.009657  0.024202  0.011342  0.009174  0.009983  0.031785   
2020-07-29      0.008502  0.025779  0.005607  0.018182  0.026359  0.007109   

Province_State                                          ...   Wyoming         \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park Platte   
2020-07-27      0.005602  0.075279  0.003866  0.056701  ...  0.020202    0.0   
2020-07-28      0.005571  0.042351  0.014121  0.004878  ...  0.059406    0.0   
2020-07-29      0.008310  0.038972  0.007595  0.024272  ...  0.009346    0.0   

Province_State                                                                \
Admin2          Sheridan  Sublette Sweetwater     Teton     Uinta Unassigned   
2020-07-27      0.048780  0.000000   0.027907  0.046595  0.008584        NaN   
2020-07-28      0.023256  0.033333   0.031674  0.065068  0.034043        NaN   
2020-07-29      0.022727  0.000000   0.026316  0.022508  0.008230        NaN   

Province_State                  
Admin2         Washakie Weston  
2020-07-27          0.0   0.25  
2020-07-28          0.0   0.00  
2020-07-29          0.0   0.00  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-07-27,0.012220,0.050000,NaN,-0.000303,0.017456,0.005445,0.002787,0.001133,0.000000,NaN,...,0.011375,0.125493,0.025547,0.0,0.000000,0.001925,0.011326,0.009709,0.001121,0.00
2020-07-28,0.000000,0.047619,NaN,0.031477,0.049020,0.021780,0.004447,0.001132,0.001727,NaN,...,0.021472,0.035008,0.017794,0.0,0.000000,0.006244,0.019763,0.038462,0.014558,0.04
2020-07-29,0.031522,0.000000,NaN,0.013498,0.014019,0.026386,0.008301,0.000452,0.001724,NaN,...,0.021021,0.047353,0.020979,0.0,0.142857,0.014320,0.004522,0.027778,0.005519,0.00


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                  \
Admin2         Autauga   Baldwin Barbour Bibb Blount Bullock    Butler   
2020-07-27         0.0  0.000000     0.0  0.0    0.0     0.0  0.029412   
2020-07-28         0.0  0.000000     0.0  0.0    0.0     0.0  0.000000   
2020-07-29         0.0  0.176471     0.0  0.0    2.0     0.0  0.000000   

Province_State                             ... Wyoming                  \
Admin2         Calhoun  Chambers Cherokee  ...    Park Platte Sheridan   
2020-07-27         0.0  0.027027      0.0  ...     NaN    NaN      NaN   
2020-07-28         0.0  0.000000      0.0  ...     NaN    NaN      NaN   
2020-07-29         0.0  0.000000      0.0  ...     NaN    NaN      NaN   

Province_State                                                             
Admin2         Sublette Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-07-27          NaN        NaN   NaN   NaN   0.000000      NaN    NaN  
2020-07-28          NaN        NaN   NaN   NaN   0.041667      NaN    NaN  
2020-07-29          NaN        NaN   NaN   NaN   0.000000      NaN    NaN  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-27,0.002855,0.023771,0.022892,0.006249,0.024164,0.029338,0.030488,0.005963,0.028938,0.005292,...,0.002390,0.011534,0.030853,0.036836,0.016896,0.018153,1.616879e-12,0.005277,0.036761,0.063062
2020-07-28,0.002875,0.023873,0.022921,0.003125,0.038398,0.014669,0.032981,0.006177,0.023585,0.005540,...,0.002111,0.012423,0.026978,0.036650,0.025849,0.024000,8.084397e-13,0.006187,0.067809,0.052426
2020-07-29,0.002854,0.022743,0.022189,0.007626,0.058199,0.036404,0.032761,0.007181,0.034768,0.006953,...,0.002382,0.014011,0.033905,0.036037,0.027499,0.027816,4.042199e-13,0.005442,0.058595,0.037218


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-27,0.009939,0.039179,0.007878,4.458307e-15,0.038849,4.968899e-32,0.033306,0.008956,0.041937,0.000890,...,0.000575,0.015219,0.043384,0.029771,NaN,0.029649,0.0,0.009703,0.009890,0.048821
2020-07-28,0.005363,0.033478,0.008668,2.229154e-15,0.092595,2.484450e-32,0.036267,0.010104,0.047915,0.000445,...,0.001586,0.007609,0.034088,0.032009,NaN,0.021235,0.0,0.005887,0.012088,0.079966
2020-07-29,0.003075,0.023496,0.009445,1.114577e-15,0.056936,1.242225e-32,0.035277,0.007834,0.060889,0.002326,...,0.001696,0.003805,0.045270,0.032561,NaN,0.016947,0.0,0.003976,0.020128,0.052483


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-27,0.006371,0.021008,0.027268,7.160014e-09,0.004422,0.044099,0.033632,0.010757,0.014674,0.003447,...,0.002684,0.002710,0.039898,0.056663,0.000147,0.074759,9.473903e-15,0.030607,0.171322,0.025803
2020-07-28,0.006361,0.018518,0.024145,3.580007e-09,0.051798,0.022049,0.034095,0.018355,0.023769,0.005368,...,0.001690,0.005035,0.035025,0.040180,0.005553,0.037379,4.736952e-15,0.019505,0.153156,0.070097
2020-07-29,0.003792,0.016609,0.021405,6.226668e-04,0.091688,0.026409,0.035507,0.017713,0.011989,0.006738,...,0.000845,0.007215,0.055389,0.031174,0.002776,0.162746,2.368476e-15,0.015110,0.090663,0.269320


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-07-27,0.021974,0.048150,NaN,0.014036,0.020558,0.014434,0.009931,0.002896,0.007580,3.126315e-41,...,0.027978,0.016604,0.012755,0.003128,0.022833,0.015248,0.014828,0.021132,0.016302,0.017235
2020-07-28,0.018698,0.044471,NaN,0.013449,0.019583,0.021014,0.013372,0.002408,0.006219,1.563157e-41,...,0.027229,0.018490,0.012183,0.002634,0.026526,0.012980,0.015703,0.020651,0.015861,0.022308
2020-07-29,0.017945,0.034690,NaN,0.013772,0.019584,0.025537,0.012007,0.005921,0.007462,7.815787e-42,...,0.022590,0.022100,0.010454,0.001673,0.026597,0.012232,0.015047,0.023152,0.016599,0.018686


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-07-27      0.018483  0.029796  0.013317  0.029746  0.033666  0.002683   
2020-07-28      0.014070  0.026999  0.012330  0.019460  0.021825  0.017234   
2020-07-29      0.011286  0.026389  0.008969  0.018821  0.024092  0.012172   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-07-27      0.006454  0.063458  0.006980  0.036826  ...  0.016476   
2020-07-28      0.006012  0.052904  0.010550  0.020852  ...  0.037941   
2020-07-29      0.007161  0.045938  0.009073  0.022562  ...  0.023643   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-07-27      0.003906  0.030782  0.068532   0.020897  0.038414  0.007313   
2020-07-28      0.001953  0.027019  0.050933   0.026286  0.051741  0.020678   
2020-07-29      0.000977  0.024873  0.025466   0.026301  0.037125  0.014454   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-07-27           -1.0  0.000401  0.125386  
2020-07-28           -1.0  0.000200  0.062693  
2020-07-29           -1.0  0.000100  0.031347  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-07-27,0.014043,0.032504,NaN,0.010368,0.013946,0.007104,0.002187,0.000648,0.006218,NaN,...,0.012716,0.076736,0.016272,1.653628e-14,0.002604,0.002357,0.007725,0.005864,0.003417,0.000276
2020-07-28,0.007022,0.040062,NaN,0.020922,0.031483,0.014442,0.003317,0.000890,0.003973,NaN,...,0.017094,0.055872,0.017033,8.268138e-15,0.001302,0.004301,0.013744,0.022163,0.008987,0.020138
2020-07-29,0.019272,0.020031,NaN,0.017210,0.022751,0.020414,0.005809,0.000671,0.002848,NaN,...,0.019057,0.051613,0.019006,4.134069e-15,0.072080,0.009310,0.009133,0.024970,0.007253,0.010069


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                                 \
Admin2           Autauga   Baldwin   Barbour      Bibb Blount   Bullock   
2020-07-27      0.000056  0.006655  0.001333  0.000061    0.0  0.001563   
2020-07-28      0.000028  0.003327  0.000666  0.000031    0.0  0.000781   
2020-07-29      0.000014  0.089899  0.000333  0.000015    1.0  0.000391   

Province_State                                              ... Wyoming  \
Admin2            Butler   Calhoun  Chambers      Cherokee  ...    Park   
2020-07-27      0.015917  0.000012  0.019538  5.218063e-12  ...     NaN   
2020-07-28      0.007959  0.000006  0.009769  2.609032e-12  ...     NaN   
2020-07-29      0.003979  0.000003  0.004884  1.304516e-12  ...     NaN   

Province_State                                                             \
Admin2         Platte Sheridan Sublette Sweetwater Teton Uinta Unassigned   
2020-07-27        NaN      NaN      NaN        NaN   NaN   NaN   0.000288   
2020-07-28        NaN      NaN      NaN        NaN   NaN   NaN   0.020977   
2020-07-29        NaN      NaN      NaN        NaN   NaN   NaN   0.010489   

Province_State                  
Admin2         Washakie Weston  
2020-07-27          NaN    NaN  
2020-07-28          NaN    NaN  
2020-07-29          NaN    NaN  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200729,AK,3442,220222,NaN,45.0,NaN,NaN,NaN,3.0,...,223664,0,0,e3c05df1cf92b9c80941cec8daaf723905e7a0dd,0,0,0,0,0,NaN
1,20200729,AL,83782,584493,NaN,1598.0,9893.0,NaN,1109.0,NaN,...,668275,47,0,a7b049cfd14780e56e6ef98fb2aba9f82fe89838,0,0,0,0,0,NaN
2,20200729,AR,40968,454637,NaN,508.0,2747.0,NaN,NaN,108.0,...,495605,6,61,e1fad45c3dbf16efc7ddd84400b3629a9c824a71,0,0,0,0,0,NaN
3,20200729,AS,0,1037,NaN,NaN,NaN,NaN,NaN,NaN,...,1037,0,0,62e46ea05669252eb426e2da4a1e822abcf86cb3,0,0,0,0,0,NaN
4,20200729,AZ,168273,733156,NaN,2424.0,10060.0,800.0,NaN,561.0,...,901429,46,666,9c7e10ce8b2fb77e7a69df1d09c60accaab1a498,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200729,AK,3442.0,220222.0,NaN,45.0,NaN,NaN,NaN,3.0,...,223664,0,0,e3c05df1cf92b9c80941cec8daaf723905e7a0dd,0,0,0,0,0,NaN
1,20200729,AL,83782.0,584493.0,NaN,1598.0,9893.0,NaN,1109.0,NaN,...,668275,47,0,a7b049cfd14780e56e6ef98fb2aba9f82fe89838,0,0,0,0,0,NaN
2,20200729,AR,40968.0,454637.0,NaN,508.0,2747.0,NaN,NaN,108.0,...,495605,6,61,e1fad45c3dbf16efc7ddd84400b3629a9c824a71,0,0,0,0,0,NaN
3,20200729,AS,0.0,1037.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1037,0,0,62e46ea05669252eb426e2da4a1e822abcf86cb3,0,0,0,0,0,NaN
4,20200729,AZ,168273.0,733156.0,NaN,2424.0,10060.0,800.0,NaN,561.0,...,901429,46,666,9c7e10ce8b2fb77e7a69df1d09c60accaab1a498,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-07-29,AK,3442,220222,NaN,45.0,NaN,NaN,NaN,3.0,NaN,...,223664,0,0,e3c05df1cf92b9c80941cec8daaf723905e7a0dd,0,0,0,0,0,NaN
2020-07-29,AL,83782,584493,NaN,1598.0,9893.0,NaN,1109.0,NaN,595.0,...,668275,47,0,a7b049cfd14780e56e6ef98fb2aba9f82fe89838,0,0,0,0,0,NaN
2020-07-29,AR,40968,454637,NaN,508.0,2747.0,NaN,NaN,108.0,387.0,...,495605,6,61,e1fad45c3dbf16efc7ddd84400b3629a9c824a71,0,0,0,0,0,NaN
2020-07-29,AS,0,1037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1037,0,0,62e46ea05669252eb426e2da4a1e822abcf86cb3,0,0,0,0,0,NaN
2020-07-29,AZ,168273,733156,NaN,2424.0,10060.0,800.0,NaN,561.0,NaN,...,901429,46,666,9c7e10ce8b2fb77e7a69df1d09c60accaab1a498,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-07-29,AK,3442.0,220222.0,NaN,45.0,NaN,NaN,NaN,3.0,NaN,...,223664,0,0,e3c05df1cf92b9c80941cec8daaf723905e7a0dd,0,0,0,0,0,NaN
2020-07-29,AL,83782.0,584493.0,NaN,1598.0,9893.0,NaN,1109.0,NaN,595.0,...,668275,47,0,a7b049cfd14780e56e6ef98fb2aba9f82fe89838,0,0,0,0,0,NaN
2020-07-29,AR,40968.0,454637.0,NaN,508.0,2747.0,NaN,NaN,108.0,387.0,...,495605,6,61,e1fad45c3dbf16efc7ddd84400b3629a9c824a71,0,0,0,0,0,NaN
2020-07-29,AS,0.0,1037.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1037,0,0,62e46ea05669252eb426e2da4a1e822abcf86cb3,0,0,0,0,0,NaN
2020-07-29,AZ,168273.0,733156.0,NaN,2424.0,10060.0,800.0,NaN,561.0,NaN,...,901429,46,666,9c7e10ce8b2fb77e7a69df1d09c60accaab1a498,0,0,0,0,0,NaN


# TODO - CONTINUE FROM HERE